### Setup a Conda Environment on Coiled

In [ ]:
import coiled

coiled.create_software_environment(
   name='mapshader-tiling',
   conda={
       'channels': ['conda-forge', 'defaults'],
       'dependencies': [
           'python=3.9',
           'mapshader',
           'dask=2022.04.2',
           'distributed=2022.4.2',
           'cloudpickle=2.0.0',
           'spatialpandas',
           'boto3',
       ],
    },
)

### Create Dask Cluster on Coiled 

In [ ]:
from coiled.v2 import Cluster
cluster = Cluster(name='mapshader-tiler',
                  n_workers=10,
                  worker_cpu=2,
                  worker_options={"nthreads": 1},
                  scheduler_memory="8 GiB",
                  software='mapshader-tiling')

from dask.distributed import Client
client = Client(cluster)
print('Dashboard:', client.dashboard_link)

### Clear cluster memory if necessary

In [ ]:
client.restart()

## Tile World Countries

### Setup Mapshader Source

In [ ]:
import geopandas as gpd
import mapshader
import spatialpandas

from mapshader.sources import VectorSource


def world_countries_source():

    # construct transforms
    select_by_attrs_transform = dict(name='select_by_attributes',
                                     args=dict(field='name',
                                               value=['Antarctica', 'Fr. S. Antarctic Lands'],
                                               operator='NOT IN'))

    reproject_transform = dict(name='reproject_vector', args=dict(epsg=3857))
    buffered_extent_transform = dict(name='add_projected_buffered_extent', 
                                     args=dict(crs='4326',
                                               buffer_distance=.01,
                                               geometry_field='geometry'))
    sp_transform = dict(name='to_spatialpandas', args=dict(geometry_field='geometry'))
    
    transforms = [select_by_attrs_transform,
                  reproject_transform,
                  buffered_extent_transform,
                  sp_transform]

    # construct value obj
    source_obj = dict()
    source_obj['name'] = 'World Countries'
    source_obj['key'] = 'world-countries'
    source_obj['text'] = 'World Countries'
    source_obj['description'] = 'World Country Polygons'
    source_obj['geometry_type'] = 'polygon'
    source_obj['agg_func'] = 'max'
    source_obj['shade_how'] = 'linear'
    source_obj['span'] = 'min/max'
    source_obj['raster_interpolate'] = 'linear'
    source_obj['xfield'] = 'x'
    source_obj['yfield'] = 'y'
    source_obj['zfield'] = 'pop_est'
    source_obj['filepath'] = gpd.datasets.get_path('naturalearth_lowres')
    source_obj['transforms'] = transforms
    source_obj['service_types'] = ['tile']
    source_obj['tiling'] = dict(min_zoom=1,
                                max_zoom=2,
                                xmin_field='buffer_0_4326_xmin',
                                xmax_field='buffer_0_4326_xmax',
                                ymin_field='buffer_0_4326_ymin',
                                ymax_field='buffer_0_4326_ymax'
                               )

    return source_obj


countries_source = VectorSource.from_obj(world_countries_source())
countries_source.load()
countries_source.data

### Generate tile images and save to an S3 bucket

In [ ]:
from mapshader.tile_utils import save_tiles_to_outpath

save_tiles_to_outpath(countries_source, outpath='s3://mapshader-tiling-test-999/countries/')

### View tiles on OSM basemap

In [ ]:
from ipyleaflet import Map, TileLayer, basemaps, basemap_to_tiles

tiles_url = 'https://mapshader-tiling-test-999.s3.amazonaws.com/countries/{z}/{x}/{y}.png'
tile_layer=TileLayer(url=tiles_url)

from ipyleaflet import Map, basemaps, basemap_to_tiles

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(48.204793, 350.121558),
    zoom=3
    )
m

m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    zoom=4,
    scroll_wheel_zoom=True)

m.add_layer(tile_layer)
m
display(m)